In [1]:
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bsoup
from collections import defaultdict

In [40]:
## Test cell
queries = ["packers","bears","eagles","cowboys","raiders","chiefs"]
nfl_info = defaultdict(list)

for query in queries:
    page_url = "https://www.si.com/nfl/" + query
    nfl_info = NFLscrape(nfl_info, query)
    
test_df = pd.DataFrame(nfl_info)

In [95]:
def SI_NFLscrape(nfl_info, query):
    page_url = "https://www.si.com/nfl/" + query

    response = requests.get(page_url)
    soup = bsoup(response.text, 'lxml')
    si_url = re.compile("/nfl/"+ query+"/\w+")

    for posting in set(soup.find_all('a')):
        article_contents = ""
        article_url = posting.get('href')
        try:
            if si_url.match(article_url):
                article_url = "https://www.si.com" + article_url
                article_response = requests.get(article_url)
                article_soup = bsoup(response.text, 'lxml')
                article_title = article_soup.h1
                #article_author = article_soup.find_all('p', {"class":"skpzhulc sk3x58id skh9fqbk"})
                texts = article_soup.find_all('p')
                for text in texts:
                    if text.text != "Read More": #and text.text not in [author.text for author in article_author]:
                        article_contents += text.text        
        except:
            continue
        finally:
            nfl_info["Team"].append(query)
            nfl_info["Article URL"].append(article_url)
            nfl_info["Article Title"].append(article_title)
            #nfl_info["Article Author"].append(article_author[0])
            nfl_info["Text"].append(article_contents)           
    
    return nfl_info

In [96]:
def ESPN_NFLscrape(nfl_info, query):
    page_url = "https://www.espn.com/blog/" + query

    response = requests.get(page_url)
    soup = bsoup(response.text, 'lxml')

    article_contents = ""
    article_url = page_url
    article_title = soup.find("header", {"class": "article-header"}).text
    #article_author = article_soup.find_all('div', {"class":"author has-bio"})
    texts = soup.find_all('p')
    
    for text in texts:
        article_contents += text.text        
    nfl_info["Team"].append(team_dict[query])
    nfl_info["Article URL"].append(article_url)
    nfl_info["Article Title"].append(article_title)
    #nfl_info["Article Author"].append(article_author[0])
    nfl_info["Text"].append(article_contents)           

    return nfl_info

In [97]:
queries = ["las-vegas-raiders", "kansas-city-chiefs", "chicago-bears", "green-bay-packers", "dallas-cowboys", "philadelphia-eagles"]
team_dict = {"las-vegas-raiders": "raiders", "kansas-city-chiefs":"chiefs", "chicago-bears":"bears", "green-bay-packers":"packers", "dallas-cowboys":"cowboys", "philadelphia-eagles":"eagles"}
nfl_info = defaultdict(list)

for query in queries:
    nfl_info = ESPN_NFLscrape(nfl_info, query)

In [98]:
queries = ["packers","bears","eagles","cowboys","raiders","chiefs"]
for query in queries:
    page_url = "https://www.si.com/nfl/" + query
    nfl_info = SI_NFLscrape(nfl_info, query)
    
test_df = pd.DataFrame(nfl_info)